# Profiling ANDES in Notebook

## Profiling with Python CProfiler

Before getting started, this example requires the config flag `PFlow.init_tds` to be `0`, which is the default value.

In [1]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

Passing `profile=True, no_output = True` to `run` will enable the profiler and have the results printed.

In [2]:
ss = andes.run(case_path, profile=True, routine='tds', no_output=True)

Working directory: "/home/hcui7/repos/andes/examples"


Loaded config from file "/home/hcui7/.andes/andes.rc"


Loaded generated Python code in "~/.andes/pycode".


Parsing input file "/home/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.5758 seconds.


...

 85%|████████████████████████████     | 85/100 [00:01<00:00, 72.84%/s]

 93%|██████████████████████████████▋  | 93/100 [00:01<00:00, 60.73%/s]

100%|████████████████████████████████| 100/100 [00:01<00:00, 70.31%/s]

Simulation completed in 1.4240 seconds.


         18966441 function calls (17908943 primitive calls) in 21.101 seconds

   Ordered by: cumulative time
   List reduced from 9157 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    11677    0.097    0.000   19.364    0.002 /home/hcui7/repos/andes/andes/system.py:1525(call_models)
        1    0.000    0.000   18.795   18.795 /home/hcui7/repos/andes/andes/routines/pflow.py:174(run)
       28    0.001    0.000   18.456    0.659 /home/hcui7/mambaforge/envs/a/lib/python3.9/site-packages/numba/core/dispatcher.py:402(_compile_for_args)
    32/28    0.002    0.000   18.412    0.658 /home/hcui7/mambaforge/envs/a/lib/python3.9/site-packages/numba/core/dispatcher.py:929(compile)
    12/11    0.000    0.000   17.448    1.586 /home/hcui7/mambaforge/envs/a/lib/python3.9/site-packages/numba/core/dispatcher.py:140(compile)
    12/11    0.000    0.000   17.448    1.586 /home/hcui7/mambaforge/envs/a/lib/python3.9/site-packages/numba/core/dis

## Profiling with `line_profiler`.

`line_profiler` provides line-based profiling results for functions. 

Install with `pip install line_profiler` and restart the notebook.

In [3]:
import andes
from andes.utils.paths import get_case

case_path = get_case('kundur/kundur_full.xlsx')

### Profile power flow 

Pass the function name to profile to the magic `%lprun`, followed by a call to the function itself or an upper-level function.

Results will be shown in a popup window.

In [4]:
%load_ext line_profiler

%lprun -f andes.routines.pflow.PFlow.run andes.run(case_path, no_output=True, default_config=True)

Working directory: "/home/hcui7/repos/andes/examples"


Loaded generated Python code in "~/.andes/pycode".


Parsing input file "/home/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.1504 seconds.


System internal structure set up in 0.0403 seconds.


...

2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0084 seconds.


-> Single process finished in 0.4152 seconds.


Alternatively, do

In [5]:
ss = andes.run(case_path, no_output=True, default_config=True)

Working directory: "/home/hcui7/repos/andes/examples"


Loaded generated Python code in "~/.andes/pycode".


Parsing input file "/home/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0750 seconds.


System internal structure set up in 0.0575 seconds.


...

2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0114 seconds.


-> Single process finished in 0.2266 seconds.


In [6]:
ss.reset()
%lprun -f ss.PFlow.run ss.PFlow.run()

System internal structure set up in 0.0180 seconds.


-> System connectivity check results:


  No islanded bus detected.


  A total of 1 island(s) detected.


  Each island has a slack bus correctly defined and enabled.


...

1: |F(x)| = 3.608627841


2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0077 seconds.


To dig into the Newton Raphson iteration steps, profile each step instead with:

In [7]:
ss.reset()
%lprun -f ss.PFlow.nr_step ss.PFlow.run()

System internal structure set up in 0.0149 seconds.


-> System connectivity check results:


  No islanded bus detected.


  A total of 1 island(s) detected.


  Each island has a slack bus correctly defined and enabled.


...

1: |F(x)| = 3.608627841


2: |F(x)| = 0.1701107882


3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103977e-07


Converged in 5 iterations in 0.0068 seconds.


### Profile time-domain simulation

In [8]:
%lprun -f ss.TDS.itm_step ss.TDS.run()


-> Time Domain Simulation Summary:
Sparse Solver: KLU
Simulation time: 0-20.0 s.
Fixed step size: h=33.33 ms. Shrink if not converged.


Initialization for dynamics was successful in 0.0308 seconds.


  0%|                                          | 0/100 [00:00<?, ?%/s]

<Toggler 1>: Line.Line_8 status changed to 0 at t=2.0 sec.
 10%|███▏                            | 10/100 [00:00<00:00, 205.20%/s]

 13%|████▏                           | 13/100 [00:00<00:00, 120.07%/s]

...

 80%|██████████████████████████▍      | 80/100 [00:01<00:00, 57.03%/s]

 87%|████████████████████████████▋    | 87/100 [00:01<00:00, 59.23%/s]

 94%|███████████████████████████████  | 94/100 [00:01<00:00, 55.48%/s]

100%|████████████████████████████████| 100/100 [00:01<00:00, 54.95%/s]

Simulation completed in 1.8206 seconds.


## Cleanup

In [9]:
!andes misc -C


    _           _         | Version 1.4.2.post33.dev0+g135cb3eb
   /_\  _ _  __| |___ ___ | Python 3.9.6 on Linux, 09/19/2021 05:23:34 PM
  / _ \| ' \/ _` / -_|_-< | 
 /_/ \_\_||_\__,_\___/__/ | This program comes with ABSOLUTELY NO WARRANTY.

"/home/hcui7/repos/andes/examples/kundur_full_out.txt" removed.
"/home/hcui7/repos/andes/examples/kundur_full_out.npz" removed.
"/home/hcui7/repos/andes/examples/kundur_full_out.lst" removed.
